In [1]:
#import statements
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

#from keypass import NOAA_api
import pylab as pl
from IPython import display

from functools import partial
import pyproj
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,explained_variance_score
from numpy import absolute,mean,std

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interactive


### $TCO_2$ was filled in. Now load the dataset for some visuals. 

Every 25th row is saved to make the dashboard useable (ie faster). 

Len before 1218966 and after trim 48759

there are 33 cruises with <100 datapoints out of 936 cruises so I consider this data loss acceptable 

In [2]:
#df =pd.read_csv('C:\\Users\\ossan\\DATA606\\DATA\\TCO2_filled.csv',index_col=0) #ORIGINAL
#plotly is too slow with complete set -use this for now 
url = 'https://raw.githubusercontent.com/ossana1/voila/main/TCO2_small.csv'
df = pd.read_csv(url,index_col=0)
dftrim = df 
print(len(df))
df['year'] =pd.to_numeric( df['year'])
df['year'] =df['year'].astype(int)
g=df.groupby('year')

48759


In [3]:
df

,cruise,station,cast,year,month,day,latitude,longitude,bottomdepth,maxsampdepth,...,aou,nitrate,nitrite,silicate,phosphate,talk,phts25p0,phtsinsitutp,tco2,Fill
851,3.0,280.0,1.0,1987,7.0,8.0,81.5750,31.5430,1028.0,962.0,...,17.206000,9.19,0.05,4.010000,0.660000,2342.031,7.693095,8.086513,2058.000,False
922,3.0,287.0,2.0,1987,7.0,10.0,81.6830,30.9980,2724.0,2621.0,...,13.380000,8.70,0.04,3.720000,0.610000,2344.678,7.698901,8.094590,2071.000,False
947,3.0,287.0,1.0,1987,7.0,10.0,81.6830,30.9980,2724.0,2621.0,...,39.269000,13.58,0.00,8.010000,0.840000,2311.641,7.726319,8.015413,2151.000,False
979,3.0,296.0,1.0,1987,7.0,12.0,81.8200,31.5850,3014.0,3001.0,...,16.665000,11.53,0.20,4.690000,0.710000,2303.184,7.676431,8.072016,2122.000,False
1005,3.0,296.0,2.0,1987,7.0,12.0,81.8200,31.5850,3014.0,3001.0,...,52.876000,14.56,0.00,10.850000,0.930000,2312.209,7.727103,8.000609,2148.000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252826,2103.0,6336.0,1.0,2018,9.0,9.0,-0.5110,165.0067,4502.0,4492.0,...,206.821167,36.81,0.00,57.875320,2.606414,2329.305,7.520553,7.766344,2261.920,True
1252887,2103.0,6338.0,1.0,2018,9.0,10.0,-1.5017,164.9982,4522.2,4512.2,...,8.933577,1.02,1.12,1.470649,0.367447,2326.474,8.075967,8.011473,2014.647,True
1252912,2103.0,6338.0,1.0,2018,9.0,10.0,-1.5017,164.9982,4522.2,4512.2,...,191.483683,36.37,0.00,141.749200,2.535400,2419.898,7.576063,7.780276,2331.261,True
1252973,2103.0,6340.0,1.0,2018,9.0,11.0,-2.4955,165.0000,2429.0,2414.8,...,193.844666,36.11,0.00,68.074720,2.593791,2331.485,7.533646,7.777696,2267.043,True


In [4]:
#dftrim = df.iloc[::25,:]
#print('len before '+ str(len(df))+' and after trim for plotly '+str(len(dftrim)))
#dftrim.to_csv( 'C:\\Users\\ossan\\DATA606\\DATA\\TCO2_small.csv') #save metrics 

In [5]:
frames = {}
for x,y in g:
    frames[x] = y
frames[list(frames.keys())[-1]]

,cruise,station,cast,year,month,day,latitude,longitude,bottomdepth,maxsampdepth,...,aou,nitrate,nitrite,silicate,phosphate,talk,phts25p0,phtsinsitutp,tco2,Fill
728492,2013.0,0.0,0.0,2019,4.0,8.0,-49.9129,-59.1499,203.0,152.4,...,23.593070,20.49,0.233650,5.38,1.46,2261.700,7.752192,7.972494,2125.584175,True
728517,2013.0,2.0,0.0,2019,4.0,9.0,-47.6833,-54.4190,5788.0,997.8,...,-0.716569,7.66,0.233200,1.39,0.59,2275.946,7.900400,8.102444,2064.073028,True
728542,2013.0,3.0,0.0,2019,4.0,10.0,-47.1050,-53.8636,6049.0,5503.6,...,-0.998145,4.95,0.182200,1.06,0.46,2288.400,7.922200,8.108184,2060.617526,True
728567,2013.0,4.0,0.0,2019,4.0,10.0,-46.5194,-53.3412,6073.0,1998.0,...,-1.905010,11.91,0.218300,1.73,0.87,2269.563,7.862100,8.075847,2075.120080,True
728592,2013.0,5.0,0.0,2019,4.0,10.0,-45.9187,-52.7895,6028.0,6023.6,...,50.785000,14.21,0.109699,4.44,1.04,2299.800,7.778500,7.984506,2136.160490,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730217,2013.0,76.0,0.0,2019,5.0,13.0,11.4034,-24.2720,5470.0,4702.5,...,81.781252,21.48,0.017500,34.71,1.42,2334.000,7.733800,7.951993,2185.196000,True
730242,2013.0,77.0,0.0,2019,5.0,15.0,20.2982,-23.4750,4422.0,4000.6,...,88.356461,21.58,0.017100,35.42,1.44,2342.300,7.731400,7.947419,2192.326000,True
730267,2013.0,78.0,0.0,2019,5.0,16.0,21.3681,-22.5370,4570.0,2000.4,...,95.539866,21.94,0.019200,24.80,1.47,2324.361,7.715000,7.951979,2184.546000,True
730292,2013.0,79.0,0.0,2019,5.0,16.0,22.4426,-21.5971,4487.0,3999.6,...,87.099036,22.53,0.017000,45.23,1.50,2352.100,7.736200,7.920496,2196.376000,True


In [6]:

var = df.columns 
years = list(frames.keys())
list_updatemenus=[];temp_dict={}
for n, year in enumerate(years):
    visible = [False] * len(years)
    visible[n] = True
    temp_dict = dict(label = str(year),
                 method = 'update',
                 args = [{'visible': visible},
                         {'title': 'Year %d' % year}])

    list_updatemenus.append(temp_dict)
    
list_c=[];c_dict={}
for n, year in enumerate(var):
    visible = [False] * len(var)
    visible[n] = True
    c_dict = dict(label = str(var),
                 method = 'update',
                 args = [{'visible': visible},
                         {'title': var}])

    list_c.append(c_dict)
    

In [7]:
#fig5 = go.Figure()
col=['Cruise', 'Station',  'Year', 'Month', 'Day', 'Bottom Depth', 'Max Sample Depth', 'Pressure', 'Depth',
       'Potential Temperature', 'theta', 'Salinity', 'Potential Temp 0m', 'Potential Temp 1km', 'Potential Temp 2km',
       'Potential Temp 3km', 'Neutral Density', 'oxygen', 'Actual O2 Utilization', 'Nitrate', 'Nitrite', 'Silicate',
       'Phosphate', 'Total Alkalinity', 'pH at STP', 'pH in Situ', 'tco2']

actu=['cruise', 'station', 'year', 'month', 'day', 'bottomdepth', 'maxsampdepth', 'pressure', 'depth',
       'temperature', 'theta', 'salinity', 'sigma0', 'sigma1', 'sigma2',
       'sigma3', 'gamma', 'oxygen', 'aou', 'nitrate', 'nitrite', 'silicate',
       'phosphate', 'talk', 'phts25p0', 'phtsinsitutp', 'tco2']
def plot_compare_lag(var, year):

    #data = te(s,BalancingAuthority)
    fig5 =go.Figure(go.Scattergeo(locationmode ='ISO-3',lon=dftrim["longitude"],
                                  lat=dftrim["latitude"],#color=var,
        mode ='markers',marker = dict(
            size = 2, color = dftrim[actu[col.index(var)]], colorscale = 'Inferno',
            cmax = df[actu[col.index(var)]].max(),colorbar_title=var) ),
    layout= dict(title ='Variable: ' +var + ' | Year:'  + str(year.astype(str)[0:4]))  )
    fig5.show()
    return fig5 

interactive(plot_compare_lag, var=col, 
            year=list(np.linspace(1984,2019,(2019-1983))),  )


interactive(children=(Dropdown(description='var', options=('Cruise', 'Station', 'Year', 'Month', 'Day', 'Botto…